# TensorFlow变量（tf.Variable）

在TensorFlow中，`tf.Variable`是存储可变状态的核心数据结构。所有模型参数（权重和偏置）都是`tf.Variable`对象。本notebook深入讲解变量的创建、操作和在模型训练中的作用。

## 学习目标
1. 理解tf.Variable与tf.constant的区别
2. 掌握变量的创建和初始化方法
3. 学会变量的原地修改操作
4. 理解变量在梯度计算中的作用
5. 掌握变量的作用域和命名

## 1. 环境设置

In [ ]:
import tensorflow as tf
import numpy as np

# 设置随机种子
RANDOM_SEED = 42
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"TensorFlow版本: {tf.__version__}")

## 2. tf.Variable vs tf.constant

| 特性 | tf.constant | tf.Variable |
|-----|------------|-------------|
| 可变性 | 不可变 | 可变 |
| 用途 | 固定数据、超参数 | 模型权重、可学习参数 |
| 梯度追踪 | 需显式指定 | 默认被追踪 |
| 内存 | 可被优化释放 | 持久存储 |

In [ ]:
# tf.Variable基本创建

# 创建一个2x3的变量
var = tf.Variable([[1, 2, 3], [4, 5, 6]])
print("变量内容:")
print(var)
print(f"\n类型: {type(var)}")
print(f"形状: {var.shape}")
print(f"数据类型: {var.dtype}")

In [ ]:
# tf.Variable与tf.constant的核心区别

# constant: 不可变
const = tf.constant([1, 2, 3])
print(f"常量: {const}")

# variable: 可变
var = tf.Variable([1, 2, 3])
print(f"变量(初始): {var}")

# 修改变量值
var.assign([4, 5, 6])
print(f"变量(修改后): {var}")

# 尝试修改常量会报错
# const.assign([4, 5, 6])  # AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

## 3. 变量的创建方式

### 3.1 基本创建

In [ ]:
# 从不同数据源创建变量

# 从Python列表
var_from_list = tf.Variable([1.0, 2.0, 3.0])
print(f"从列表创建: {var_from_list}")

# 从NumPy数组
np_array = np.array([[1, 2], [3, 4]], dtype=np.float32)
var_from_numpy = tf.Variable(np_array)
print(f"从NumPy创建: {var_from_numpy}")

# 从tf.constant
const = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)
var_from_const = tf.Variable(const)
print(f"从常量创建: {var_from_const}")

# 从另一个变量
var_copy = tf.Variable(var_from_list)
print(f"从变量复制: {var_copy}")

### 3.2 指定数据类型和名称

In [ ]:
# 指定dtype和name

# 指定数据类型
var_f64 = tf.Variable([1.0, 2.0], dtype=tf.float64)
print(f"float64变量: {var_f64.dtype}")

# 指定名称（用于调试和TensorBoard）
weights = tf.Variable(
    tf.random.normal([3, 2]),
    name="layer1_weights",
    dtype=tf.float32
)
print(f"\n命名变量: {weights.name}")
print(f"权重形状: {weights.shape}")
print(f"权重值:\n{weights}")

### 3.3 trainable参数

`trainable`参数决定变量是否参与梯度计算和优化器更新。

In [ ]:
# trainable参数演示

# 可训练变量（默认）
trainable_var = tf.Variable([1.0, 2.0, 3.0], trainable=True)
print(f"可训练变量: trainable={trainable_var.trainable}")

# 不可训练变量（如批归一化的移动平均）
non_trainable_var = tf.Variable([1.0, 2.0, 3.0], trainable=False)
print(f"不可训练变量: trainable={non_trainable_var.trainable}")

# 在梯度计算中的区别
with tf.GradientTape() as tape:
    y1 = trainable_var * 2
    y2 = non_trainable_var * 2

grad1 = tape.gradient(y1, trainable_var)
print(f"\n可训练变量的梯度: {grad1}")

with tf.GradientTape() as tape:
    y2 = non_trainable_var * 2
grad2 = tape.gradient(y2, non_trainable_var)
print(f"不可训练变量的梯度: {grad2}")

## 4. 变量的修改操作

变量支持多种原地修改方法，这些操作不会创建新的张量。

In [ ]:
# 变量修改方法

var = tf.Variable([1.0, 2.0, 3.0, 4.0])
print(f"初始值: {var.numpy()}")

# assign: 完全替换
var.assign([5.0, 6.0, 7.0, 8.0])
print(f"assign后: {var.numpy()}")

# assign_add: 原地加法
var.assign_add([1.0, 1.0, 1.0, 1.0])
print(f"assign_add后: {var.numpy()}")

# assign_sub: 原地减法
var.assign_sub([2.0, 2.0, 2.0, 2.0])
print(f"assign_sub后: {var.numpy()}")

In [ ]:
# 部分更新操作

var = tf.Variable([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=tf.float32)
print(f"初始值:\n{var.numpy()}\n")

# 使用索引修改单个元素
var[0, 0].assign(100.0)
print(f"修改[0,0]后:\n{var.numpy()}\n")

# 使用切片修改多个元素
var[1, :].assign([40.0, 50.0, 60.0])
print(f"修改第2行后:\n{var.numpy()}")

## 5. 变量与梯度计算

`tf.Variable`是深度学习训练的核心，因为`tf.GradientTape`默认追踪所有`tf.Variable`的操作。

In [ ]:
# 梯度计算示例

# 定义可训练参数
w = tf.Variable(2.0)
b = tf.Variable(1.0)

# 输入数据
x = tf.constant(3.0)
y_true = tf.constant(7.0)  # 真实标签

# 前向传播和损失计算
with tf.GradientTape() as tape:
    y_pred = w * x + b
    loss = tf.square(y_pred - y_true)

print(f"输入 x: {x.numpy()}")
print(f"预测值: {y_pred.numpy()}")
print(f"真实值: {y_true.numpy()}")
print(f"损失: {loss.numpy()}")

# 计算梯度
gradients = tape.gradient(loss, [w, b])
print(f"\ndL/dw: {gradients[0].numpy()}")
print(f"dL/db: {gradients[1].numpy()}")

In [ ]:
# 模拟一次训练步骤

# 参数初始化
w = tf.Variable(2.0)
b = tf.Variable(1.0)
learning_rate = 0.01

# 训练数据
x = tf.constant(3.0)
y_true = tf.constant(7.0)

print(f"训练前: w={w.numpy()}, b={b.numpy()}")

# 训练步骤
for step in range(5):
    with tf.GradientTape() as tape:
        y_pred = w * x + b
        loss = tf.square(y_pred - y_true)
    
    gradients = tape.gradient(loss, [w, b])
    
    # 手动更新参数
    w.assign_sub(learning_rate * gradients[0])
    b.assign_sub(learning_rate * gradients[1])
    
    print(f"Step {step+1}: loss={loss.numpy():.4f}, w={w.numpy():.4f}, b={b.numpy():.4f}")

## 6. 在Keras模型中的变量

In [ ]:
# Keras层自动管理变量

# 创建简单模型
model = tf.keras.Sequential([
    tf.keras.layers.Dense(4, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(2)
])

# 构建模型
model.build()

# 查看所有变量
print("模型所有变量:")
for var in model.variables:
    print(f"  {var.name}: shape={var.shape}, trainable={var.trainable}")

print(f"\n可训练变量数量: {len(model.trainable_variables)}")
print(f"不可训练变量数量: {len(model.non_trainable_variables)}")

In [ ]:
# 自定义层中的变量管理

class CustomDense(tf.keras.layers.Layer):
    """自定义全连接层，展示变量创建"""
    
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = units
    
    def build(self, input_shape):
        # 创建可训练权重
        self.kernel = self.add_weight(
            name='kernel',
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',
            trainable=True
        )
        # 创建可训练偏置
        self.bias = self.add_weight(
            name='bias',
            shape=(self.units,),
            initializer='zeros',
            trainable=True
        )
        # 创建不可训练的计数器
        self.call_count = self.add_weight(
            name='call_count',
            shape=(),
            initializer='zeros',
            trainable=False
        )
        super().build(input_shape)
    
    def call(self, inputs):
        self.call_count.assign_add(1.0)
        return tf.matmul(inputs, self.kernel) + self.bias

# 测试自定义层
layer = CustomDense(5)
output = layer(tf.random.normal([2, 3]))

print("自定义层变量:")
for var in layer.variables:
    print(f"  {var.name}: trainable={var.trainable}")

# 多次调用后检查计数器
_ = layer(tf.random.normal([2, 3]))
_ = layer(tf.random.normal([2, 3]))
print(f"\n层被调用次数: {layer.call_count.numpy():.0f}")

## 7. 变量的设备放置

In [ ]:
# 查看变量所在设备

var = tf.Variable([1.0, 2.0, 3.0])
print(f"变量设备: {var.device}")

# 在特定设备上创建变量
with tf.device('/CPU:0'):
    cpu_var = tf.Variable([1.0, 2.0], name='cpu_var')
    print(f"CPU变量设备: {cpu_var.device}")

# 如果有GPU
if tf.config.list_physical_devices('GPU'):
    with tf.device('/GPU:0'):
        gpu_var = tf.Variable([1.0, 2.0], name='gpu_var')
        print(f"GPU变量设备: {gpu_var.device}")

## 8. 变量的保存和加载

In [ ]:
import tempfile
import os

# 使用tf.train.Checkpoint保存变量

# 创建变量
w = tf.Variable([1.0, 2.0, 3.0], name='weights')
b = tf.Variable([0.0], name='bias')

print(f"保存前: w={w.numpy()}, b={b.numpy()}")

# 创建检查点
checkpoint = tf.train.Checkpoint(weights=w, bias=b)

# 保存到临时目录
with tempfile.TemporaryDirectory() as tmpdir:
    save_path = checkpoint.save(os.path.join(tmpdir, 'ckpt'))
    print(f"保存路径: {save_path}")
    
    # 修改变量值
    w.assign([10.0, 20.0, 30.0])
    b.assign([5.0])
    print(f"修改后: w={w.numpy()}, b={b.numpy()}")
    
    # 恢复变量
    checkpoint.restore(save_path)
    print(f"恢复后: w={w.numpy()}, b={b.numpy()}")

## 9. 最佳实践

In [ ]:
# 最佳实践示例

# 1. 使用有意义的名称
def create_layer_variables(input_dim, output_dim, layer_name):
    """创建带有清晰命名的层变量"""
    weights = tf.Variable(
        tf.random.normal([input_dim, output_dim], stddev=0.1),
        name=f'{layer_name}/weights'
    )
    bias = tf.Variable(
        tf.zeros([output_dim]),
        name=f'{layer_name}/bias'
    )
    return weights, bias

w1, b1 = create_layer_variables(10, 5, 'hidden_layer')
print(f"权重名称: {w1.name}")
print(f"偏置名称: {b1.name}")

# 2. 使用适当的初始化
print("\n常用初始化方法:")
print(f"Xavier/Glorot: {tf.keras.initializers.GlorotUniform()(shape=(2, 2))}")
print(f"He: {tf.keras.initializers.HeNormal()(shape=(2, 2))}")

## 知识点总结

### 变量操作速查表

| 操作 | 方法 | 说明 |
|-----|------|------|
| 创建变量 | `tf.Variable(initial_value)` | 初始值必须提供 |
| 完全赋值 | `var.assign(value)` | 替换所有值 |
| 原地加法 | `var.assign_add(value)` | var += value |
| 原地减法 | `var.assign_sub(value)` | var -= value |
| 索引修改 | `var[i].assign(value)` | 修改特定位置 |
| 读取值 | `var.numpy()` | 转换为NumPy数组 |

### 关键要点

1. **tf.Variable是可变的，tf.constant是不可变的**
2. **trainable参数控制是否参与梯度计算**
3. **tf.GradientTape默认追踪所有tf.Variable**
4. **使用assign系列方法进行原地修改**
5. **Keras层使用add_weight自动管理变量**
6. **使用tf.train.Checkpoint保存和恢复变量**
7. **为变量命名有助于调试和TensorBoard可视化**